In [1]:
#SETUP
PATH = "/Users/habonissa/WHOLE BRAIN CFOS/Dams/" #Path to folder containing .csv files you want analyzed
import os
os.mkdir(PATH+'Analysis') #Make an analysis folder to put results from this code into

In [2]:
#EXTRACT NAMES AND TOTAL NUMBER OF FILES TO ANALYZE FROM PATH
filenames = []
filecount = 0
for i in range (0, len(os.listdir(PATH))):
    if ".csv" in os.listdir(PATH)[i]: #Note: don't add .csv files to PATH folder that you don't want analyzed
        filenames.append(os.listdir(PATH)[i])
        filecount = filecount +1

In [3]:
#IMPORT DATA FROM ALL RELEVANT FILES
import pandas as pd
data = {} #using dictionary to make unique names for each file, can use data.keys() to see mom number and conditions
for i in range(0,filecount):
    mom_number = filenames[i][3]
    condition = filenames[i][5]
    data[mom_number + condition] = pd.read_csv(PATH+filenames[i])
    data[mom_number + condition] =  data[mom_number + condition].rename(columns={"count":"cells", "volume (mm^3)":"volume", "density (cells/mm^3)":"density"})

In [4]:
#REORGANIZE ALL DATA INTO SINGLE WELL-LABELLED DATAFRAME FOR PLOTTING AND ANALYSIS
ID = []
condition = []
roi = []
density = []
name = []
#fill the columns individually
for i in range(0,filecount):
    for k in range(0,len(data[list(data.keys())[1][0]+list(data.keys())[1][1]])):
        density.append(data[list(data.keys())[i][0] + list(data.keys())[i][1]].density[k]) 
        roi.append(data[list(data.keys())[i][0] + list(data.keys())[i][1]].acronym[k])
        name.append(data[list(data.keys())[i][0] + list(data.keys())[i][1]].name[k])
        ID.append(list(data.keys())[i][0])
        condition.append(list(data.keys())[i][1])
#make each column into a dataframe to concatenate them, analyze them and retain labels
ID = pd.DataFrame(ID, columns = ['ID'])
condition = pd.DataFrame(condition, columns = ['condition'])
roi = pd.DataFrame(roi, columns = ['roi'])
density = pd.DataFrame(density, columns = ['density'])
name = pd.DataFrame(name,columns = ['name'])
densityData = pd.concat([ID, condition,roi,density,name],1)
#add hemisphere data for easy parsing
hemisphere = []
finalLen = (len(data[list(data.keys())[1][0]+list(data.keys())[1][1]])*filecount)
for i in range(0,finalLen):
    if '-L' in densityData.roi[i]:
        hemisphere.append('Left')
    elif '-R' in densityData.roi[i]:
        hemisphere.append('Right')
    else:
        hemisphere.append('na')
hemisphere = pd.DataFrame(hemisphere,columns=['hemisphere'])
densityData = pd.concat([densityData,hemisphere],1)
#get rid of suffixes for rois and names
densityData["roi"] = densityData["roi"].str.replace("-L", "")
densityData["roi"] = densityData["roi"].str.replace("-R", "")
densityData["name"] = densityData["name"].str.replace("left", "")
densityData["name"] = densityData["name"].str.replace("right", "")
densityData.head()

,ID,condition,roi,density,name,hemisphere
0,6,n,bkd,46.167821,background,na
1,6,n,root,479.402077,root,Left
2,6,n,grey,507.758620,Basic cell groups and regions,Left
3,6,n,CH,658.637687,Cerebrum,Left
4,6,n,CTX,700.221960,Cerebral cortex,Left


In [ ]:
#PLOT EVERY REGION, BOTH HEMISPHERES AND TOTAL BRAIN DATA
#This will export a searchable PDF if interested in taking a quick look at each ROI
import matplotlib.pyplot as plt
import numpy as np
#first, create x variables for the two halves of the plots
xN = np.zeros(int(filecount/2)) #half in condition one -> note: assumption is that sample size is equal
xS = np.ones(int(filecount/2)) #half in condition two -> note: assumption is that sample size is equal
#next, create a list of unique names for ROIs
regions = len(densityData.roi.unique())-1 #minus one due to some error
fig, axs = plt.subplots(regions,3,sharey='row') #can also do sharey=True for all plots, but would increase runtime
fig.set_size_inches(3, 654)
#delete first row of plots that are all empty
fig.delaxes(axs[0][0])
fig.delaxes(axs[0][1])
fig.delaxes(axs[0][2])
fig.tight_layout()
for i in range(1,regions):
    #noSep, -L
    axs[i,0].scatter(xN,densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")],label="noSep",color="pink")
    avgL_n = sum(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])/len(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])
    axs[i,0].scatter(0,avgL_n,color='black')
    #Sep, -L
    axs[i,0].scatter(xS,densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")],label="Sep",color="turquoise")
    avgL_S = sum(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])/len(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])
    axs[i,0].scatter(1,avgL_S,color='black')
    #label
    axs[i,0].set_title(densityData.roi.unique()[i] + "-L")
    #noSep, -R
    axs[i,1].scatter(xN,densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")],label="noSep",color="pink") #density where conditions are met
    avgR_n = sum(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])/len(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])
    axs[i,1].scatter(0,avgR_n,color='black')
    #Sep, -R
    axs[i,1].scatter(xS,densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")],label="Sep",color="turquoise") #density where conditions are m
    avgR_S = sum(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])/len(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])
    axs[i,1].scatter(1,avgR_S,color='black')
    #label
    axs[i,1].set_title(densityData.roi.unique()[i] + "-R")
    #noSep, -T
    noSepL = list(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])
    noSepR = list(densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])
    noSepT = [ noSepL[k] + noSepR[k] for k in range (len (noSepL))] 
    noSepT = pd.DataFrame(noSepT, columns = ['noSepT'])
    noSepT = noSepT.noSepT/2
    axs[i,2].scatter(xN,noSepT,color='pink')
    noSepTAvg = sum(noSepT)
    noSepTAvg = noSepTAvg/4
    axs[i,2].scatter(0,noSepTAvg,color='black')
    #label
    axs[i,2].set_title(densityData.roi.unique()[i] + "-T")
    #Sep, -T
    SepL = list(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Left")])
    SepR = list(densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i]) & (densityData.hemisphere == "Right")])
    SepT = [ SepL[k] + SepR[k] for k in range (len (SepL))] 
    SepT = pd.DataFrame(SepT, columns = ['SepT'])
    SepT = SepT.SepT/2
    axs[i,2].scatter(xS,SepT,color='turquoise')
    SepTAvg = sum(SepT)
    SepTAvg = SepTAvg/4
    axs[i,2].scatter(1,SepTAvg,color='black')
    #label
    axs[i,2].set_title(densityData.roi.unique()[i] + "-T")
plt.savefig(PATH+'Analysis/'+'density_plots.pdf',dpi=30, bbox_inches='tight')
plt.close()

In [ ]:
#BASIC STATS FOR EVERY REGION, BOTH HEMISPHERES AND TOTAL BRAIN DATA
import scipy.stats as stats
right_pval = []
left_pval = []
total_pval = []
right_tstat = []
left_tstat = []
total_tstat = []
roi_names = list(densityData.roi.unique()[1:(len(densityData.roi.unique())-1)])
names = list(densityData.name.unique()[1:(len(densityData.name.unique())-1)])
regions = len(densityData.roi.unique())

for i in range (1,regions-1):
    right_n = densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i])& (densityData.hemisphere == "Right")]
    left_n = densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i])& (densityData.hemisphere == "Left")]
    total_n = densityData.density[(densityData.condition == "n") & (densityData.roi == densityData.roi.unique()[i])]
    right_S = densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i])& (densityData.hemisphere == "Right")]
    left_S = densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i])& (densityData.hemisphere == "Left")]
    total_S = densityData.density[(densityData.condition == "S") & (densityData.roi == densityData.roi.unique()[i])]
    t_stat_right, p_val_right = stats.ttest_ind(right_n,right_S,equal_var=True)
    t_stat_left, p_val_left = stats.ttest_ind(left_n,left_S,equal_var=True)
    t_stat_total, p_val_total = stats.ttest_ind(total_n,total_S,equal_var=True)
    right_pval.append(p_val_right)
    left_pval.append(p_val_left)
    total_pval.append(p_val_total)
    right_tstat.append(t_stat_right)
    left_tstat.append(t_stat_left)
    total_tstat.append(t_stat_total)
right_pval = pd.DataFrame(right_pval, columns = ['right_pval'])
left_pval = pd.DataFrame(left_pval, columns = ['left_pval'])
total_pval = pd.DataFrame(total_pval, columns = ['total_pval'])
right_tstat = pd.DataFrame(right_tstat, columns = ['right_tstat'])
left_tstat = pd.DataFrame(left_tstat, columns = ['left_tstat'])
total_tstat = pd.DataFrame(total_tstat, columns = ['total_tstat'])
roi_names = pd.DataFrame(roi_names, columns = ['roi_names'])
names = pd.DataFrame(names,columns=['names'])
densityPstats = pd.concat([roi_names,names, total_pval, right_pval, left_pval],1)
densityPstats['Total_Pval_Rank'] = densityPstats['total_pval'].rank(ascending=True)
densityPstats = densityPstats.set_index('Total_Pval_Rank')
densityPstats = densityPstats.sort_index()
densityPstats.to_csv(PATH+'Analysis/'+'density_Pvals.csv')